# How-to guide for Personalized Recommendations use-case on Abacus.AI platform
This notebook provides you with a hands on environment to build a model that creates personalized recommendations using the Abacus.AI Python Client Library.

We'll be using the [User Item Recommendations](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/user_movie_ratings.csv), [Movie Attributes](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/movies_metadata.csv), and [User Attributes](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/users_metadata.csv) datasets, each of which has information about the user and/or their choice of movies.


## Table of content
[Installation and imports](#scrollTo=-CHABbdhcDZg)

[1. Create a Project](#scrollTo=j_6LiH43cM9Z)

[2. Add Datasets to your Project](#scrollTo=8O41vBUQcgxN)

[3. Train a Model](#scrollTo=RWvYvPEmdfg7)

[(Checkpoint)](#scrollTo=C0mIg2VHdnfA)

[4. Evaluate your Model Metrics](#scrollTo=jBK2e1WNd6L3)

[5. Deploy Model](#scrollTo=Xc5YAK8veBt1)

[6. Make Prediction](#scrollTo=BzFpIsJ_eGmk)

## Installation and imports

1. Install the Abacus.AI library.

In [ ]:
!pip install abacusai

We'll also import pandas for visualization in this notebook.

In [ ]:
import pandas as pd  # A tool we'll use to download and preview CSV files
pd.set_option('display.max_colwidth', None)  # We set the max_colwidth to None to have an unlimited width of characters

2. Add your Abacus.AI [API Key](https://abacus.ai/app/profile/apikey) generated using the API dashboard as follows:

In [ ]:
#@title Abacus.AI API Key
api_key = ''  #@param {type: "string"}

3. Import the Abacus.AI library and instantiate a client

In [ ]:
from abacusai import ApiClient
client = ApiClient(api_key)

## 1. Create a Project

Abacus.AI projects are containers that have datasets and trained models. By specifying a business **Use Case**, Abacus.AI tailors the deep learning algorithms to produce the best performing model possible for your data.

We'll call the `list_use_cases` method to retrieve a list of the available Use Cases currently available on the Abacus.AI platform.

In [ ]:
use_cases = client.list_use_cases()
use_cases

We can use pandas to pretty-print the use cases.

In [ ]:
pd.DataFrame(use_case.to_dict() for use_case in use_cases)

In this notebook, we're going to create a model that creates personalized recommendations using the User Item Recommendations, Movie Attributes, and User Attributes datasets. The **USER_RECOMMENDATIONS** use case is best tailored for this situation. For the purpose of taking an example, we will use the IMDB movie dataset that has movie metadata, user metadata, and user-movie ratings.

In [ ]:
#@title Abacus.AI Use Case

use_case = 'USER_RECOMMENDATIONS'  #@param {type: "string"}

By calling the `describe_use_case_requirements` method we can view what datasets are required for this use_case.

In [ ]:
requirements = client.describe_use_case_requirements(use_case)
pd.DataFrame(requirement.to_dict() for requirement in requirements)

Finally, let's create the project.

In [ ]:
recommendations_project = client.create_project(name='Movie Recommendations', use_case=use_case)
recommendations_project.to_dict()

## 2. Add Datasets to your Project

Abacus.AI can read datasets directly from `AWS S3` or `Google Cloud Storage` buckets, otherwise you can also directly upload and store your datasets with Abacus.AI. For this notebook, we will have Abacus.AI read the datasets directly from a public S3 bucket's location.

We are using three datasets for this notebook. We'll tell Abacus.AI how the datasets should be used when creating them by tagging each dataset with a special Abacus.AI **Dataset Type**.
- [User Item Recommendations](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/user_movie_ratings.csv) (**USER_ITEM_INTERACTIONS**): 
This dataset contains information about multiple users' ratings of movies with specified IDs.
- [Movie Attributes](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/movies_metadata.csv) (**CATALOG_ATTRIBUTES**): This dataset contains attributes about movies with specified IDs, such as each movie's name and genre.
- [User Attributes](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/users_metadata.csv) (**USER_ATTRIBUTES**): This dataset contains information about users with specified IDs, such as their age, gender, occupation, and zip code. 

### Add the datasets to Abacus.AI

First we'll use Pandas to preview the files, then add them to Abacus.AI.

In [ ]:
pd.read_csv('https://s3.amazonaws.com//abacusai.exampledatasets/user_recommendations/user_movie_ratings.csv')

In [ ]:
pd.read_csv('https://s3.amazonaws.com//abacusai.exampledatasets/user_recommendations/movies_metadata.csv')

In [ ]:
pd.read_csv('https://s3.amazonaws.com//abacusai.exampledatasets/user_recommendations/users_metadata.csv')

Using the Create Dataset API, we can tell Abacus.AI the public S3 URI of where to find the datasets. We will also give each dataset a Refresh Schedule, which tells Abacus.AI when it should refresh the dataset (take an updated/latest copy of the dataset).

The Refresh Schedule is given with a cron string. For example, when entering "0 12 * * *", the dataset is going to be re-read from the s3 at 12pm UTC, so that no update are missed.

If you're unfamiliar with Cron Syntax, Crontab Guru can help translate the syntax back into natural language: [https://crontab.guru/#0_12_\*_\*_\*](https://crontab.guru/#0_12_*_*_*)

**Note: This cron string will be evaluated in UTC time zone**

In [ ]:
user_item_dataset = client.create_dataset_from_file_connector(
    name='User Item Recommendations',
    table_name='User_Item_Recommendations',
    location='s3://abacusai.exampledatasets/user_recommendations/user_movie_ratings.csv',
    refresh_schedule='0 12 * * *'
)

movie_attributes_dataset = client.create_dataset_from_file_connector(
    name='Movie Attributes',
    table_name='Movie_Attributes',
    location='s3://abacusai.exampledatasets/user_recommendations/movies_metadata.csv',
    refresh_schedule='0 12 * * *'
)

user_attributes_dataset = client.create_dataset_from_file_connector(
    name='User Attributes',
    table_name='User_Attributes',
    location='s3://abacusai.exampledatasets/user_recommendations/users_metadata.csv',
    refresh_schedule='0 12 * * *'
)

datasets = [user_item_dataset, movie_attributes_dataset, user_attributes_dataset]
for dataset in datasets:
    dataset.wait_for_inspection()

## 3. Create Feature Groups and add them to your Project

Datasets are created at the organization level and can be used to create feature groups as follows:

In [ ]:
feature_group = client.create_feature_group(table_name='personalized_recommendations',sql='SELECT * from User_Item_Recommendations')

Adding Feature Group to the project:

In [ ]:
client.add_feature_group_to_project(feature_group_id=feature_group.feature_group_id,project_id = recommendations_project.project_id)

Setting the Feature Group type according to the use case requirements:

In [ ]:
client.set_feature_group_type(feature_group_id=feature_group.feature_group_id, project_id = recommendations_project.project_id, feature_group_type= "USER_ITEM_INTERACTIONS")

Check current Feature Group schema:

In [ ]:
client.get_feature_group_schema(feature_group_id=feature_group.feature_group_id)

#### For each **Use Case**, there are special **Column Mappings** that must be applied to a column to fulfill use case requirements. We can find the list of available **Column Mappings** by calling the *Describe Use Case Requirements* API:

In [ ]:
client.describe_use_case_requirements(use_case)[0].allowed_feature_mappings

In [ ]:
client.set_feature_mapping(project_id = recommendations_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='movie_id',feature_mapping='ITEM_ID')
client.set_feature_mapping(project_id = recommendations_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='user_id',feature_mapping='USER_ID')
client.set_feature_mapping(project_id = recommendations_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='timestamp',feature_mapping='TIMESTAMP')

For each required Feature Group Type within the use case, you must assign the Feature group to be used for training the model:

In [ ]:
client.use_feature_group_for_training(project_id=recommendations_project.project_id, feature_group_id=feature_group.feature_group_id)

Now that we've our feature groups assigned, we're almost ready to train a model!

To be sure that our project is ready to go, let's call project.validate to confirm that all the project requirements have been met:

In [ ]:
recommendations_project.validate()

## 4. Train a Model

For each **Use Case**, Abacus.AI has a bunch of options for training. We can call the `get_training_config_options` API to see the available options.

In [ ]:
training_config_options = recommendations_project.get_training_config_options()
training_config_options

To have a nice display:

In [ ]:
pd.DataFrame(training_config_option.to_dict() for training_config_option in training_config_options)

In this notebook, we'll just train with the default options, but definitely feel free to experiment, especially if you have familiarity with Machine Learning (See the description of the parameters [here](https://abacus.ai/app/help/useCases/user_recommendations/training)).

In [ ]:
training_config = {
    'BATCH_SIZE': None,
    'DOWNSAMPLE_ITEM_POPULARITY_PERCENTILE': None,
    'DROPOUT_RATE': None,
    'EXCLUDE_TIME_FEATURES': None,
    'IGNORE_ACTION_WEIGHT_COLUMN': None,
    'MAX_HISTORY_LEN': None,
    'MAX_USER_HISTORY_LEN_PERCENTILE': None,
    'RECENT_DAYS_FOR_TRAINING': None,
    'RERANKING_PERSONALIZATION_FACTOR': None,
    'SEARCH_QUERY_COLUMN': None,
    'SKIP_HISTORY_FILTERING': False,
    'TARGET_EVENT_WEIGHTS': None,
    'TEST_ON_USER_SPLIT': False,
    'TEST_SPLIT': None,
    'TRAINING_START_DATE': None,
    'UNORDERED_HISTORY': False
 }

In [ ]:
recommendations_model = recommendations_project.train_model(training_config=training_config)
recommendations_model.to_dict()

After we start training the model, we can call this blocking call that routinely checks the status of the model until it is trained and evaluated.

In [ ]:
recommendations_model.wait_for_evaluation()

## **(Checkpoint)**
Training can take an hour or two to complete, but we encourage you to run the remaining calls on your own time. If your page times out or you hit refresh, you can restore your progress in this section.

In [ ]:
!pip install abacusai
import pandas as pd
pd.set_option('display.max_colwidth', None)
api_key = ''  #@param {type: "string"}
from abacusai import ApiClient
client = ApiClient(api_key) 
recommendations_project = next(project for project in client.list_projects() if project.name == 'Movie Recommendations')
recommendations_model = recommendations_project.list_models()[-1]
recommendations_model.wait_for_evaluation()

## Evaluate your Model Metrics

After your model is done training you can inspect the model's quality by reviewing the model's metrics.

In [ ]:
recommendations_model.get_metrics().to_dict()

To get a better understanding on what these metrics mean, visit our [documentation](https://abacus.ai/app/help/useCases/USER_RECOMMENDATIONS/training) page.

## 5. Deploy Model

After the model has been trained, we need to deploy the model to be able to start making predictions. Deploying a model will reserve cloud resources to host the model for Realtime and/or batch predictions.

In [ ]:
recommendations_deployment = client.create_deployment(name='Personalized Recommendations Deployment',model_id=recommendations_model.model_id)
recommendations_deployment.wait_for_deployment()

After the model is deployed, we need to create a deployment token for authenticating prediction requests. This token is only authorized to predict on deployments in this project, so it's safe to embed this token inside of a user-facing application or website.

In [ ]:
deployment_token = recommendations_project.create_deployment_token().deployment_token
deployment_token

## 6. Make Predictions


Now that you have an active deployment and a deployment token to authenticate requests, you can make the `get_recommendations` API call below.

To see a full description of the prediction API parameters, visit our [documentation](https://abacus.ai/app/help/useCases/USER_RECOMMENDATIONS/predictions) page. 

NB: The REST API keywords described in the documentation use the CamelCase word convention while the Python API one below use the snake case convention, see [here](https://medium.com/better-programming/string-case-styles-camel-pascal-snake-and-kebab-case-981407998841) for more information.

For the purpose of data visualization, we store the source file content in Pandas' dataframes.

In [ ]:
movies = pd.read_csv('https://s3.amazonaws.com//abacusai.exampledatasets/user_recommendations/movies_metadata.csv', dtype=object)
users = pd.read_csv('https://s3.amazonaws.com//abacusai.exampledatasets/user_recommendations/users_metadata.csv', dtype=object)

### Select a User ID

The first step is to select a user by inputting his/her user ID.

In [ ]:
user_id = "10" #@param {type: "string"}
users[users["user_id"] == user_id]

### Build the query

The query is a dictionary with the key being the column used as **ITEM_ID** (in our example, the *movie_id* column) and the value being the corresponding ID.

In [ ]:
my_query_data = {"user_id": user_id}

### Run the Get Recommendations API

This command will return a list of recommendations for the user with the specified ID. The recommendation would be determined based on what movies the user liked in the past and how the movies and users are related to each other depending on their attributes.


In [ ]:
recommendations = ApiClient().get_recommendations(
    deployment_token=deployment_token, 
    deployment_id=recommendations_deployment.deployment_id, 
    query_data=my_query_data,
)
recommendations

A convenient way to visualize the data is within a Pandas Dataframe, by joining it with the movies dataframe to have the movies' names and genres.

In [ ]:
recommendations_df = pd.DataFrame(recommendations).merge(movies, on="movie_id")
recommendations_df

### Set the number of results per page

For convenience, the number of recommendations is set to 50 per page by default. You can change the default value by editing the value of the `num_items` keyword. The example below sets the number of pages to 10.

In [ ]:
recommendations_base = ApiClient().get_recommendations(
    deployment_token=deployment_token, 
    deployment_id=recommendations_deployment.deployment_id, 
    query_data=my_query_data,
    num_items=10,
)
recommendations_base_df = pd.DataFrame(recommendations_base).merge(movies, on="movie_id")
recommendations_base_df

You can easily select the page to display with the keyword `page`. For example, let's say that the num_items is set to 10 with the total recommendations list size of 50 recommended items, then an input value of 2 in the `page` keyword will display a list of items that rank from 11th to 20th. 

In [ ]:
recommendations_page = ApiClient().get_recommendations(
    deployment_token=deployment_token, 
    deployment_id=recommendations_deployment.deployment_id, 
    query_data={"user_id":user_id},
    num_items=10,
    page=2,
)
recommendations_page_df = pd.DataFrame(recommendations_page).merge(movies, on="movie_id")
recommendations_page_df

You can add a column with the relative item scores by specifying the column name for the keyword `score_field` (example: "score")

In [ ]:
recommendations_score = ApiClient().get_recommendations(
    deployment_token=deployment_token, 
    deployment_id=recommendations_deployment.deployment_id, 
    query_data=my_query_data,
    num_items=10,
    score_field = "score"
)
recommendations_score_df = pd.DataFrame(recommendations_score).merge(movies, on="movie_id")
recommendations_score_df

### Use of Scaling Factors to bias the model toward certain items

You can use a scaling factor to add a bias toward certain items of your items datasets using the keyword `scaling_factor`. 

The input is a list of dictionaries where the format of each dictionary is as follows: {"column": "col0", "values": ["value0", "value1"], "factor": 1.1}. 

The command below is using scaling factors based on the column "Genres" to add positive bias to the comedies with a scaling factor of 3, and a negative bias to the dramas with a scaling factor of 0.25.

We now have a Comedy in the second recommendation for user ID 10 (while in the unscaled run, there was no Comedy in the first 50 recommendations). Similarly there is no more Drama|Romance in her first 10 recommendations.

In [ ]:
recommendations_scaling = ApiClient().get_recommendations(
    deployment_token=deployment_token, 
    deployment_id=recommendations_deployment.deployment_id, 
    query_data=my_query_data,
    num_items=10,
    score_field = "score",
    scaling_factors=[
        {"column": "genres", "values": ["Comedy"], "factor": 3},
        {"column": "genres", "values": ["Drama"], "factor": 0.25},
    ],
)
recommendations_scaling_df = pd.DataFrame(recommendations_scaling).merge(movies, on="movie_id")
recommendations_scaling_df

### Item exclusion

You can also exclude certain items from the list of recommendations using the keyword `exclude_items`. The command below is removing with genres Comedy|Romance and Drama|Romance from the recommendations list for the user.

In [ ]:
recommendations_excluded = ApiClient().get_recommendations(
    deployment_token=deployment_token, 
    deployment_id=recommendations_deployment.deployment_id, 
    query_data=my_query_data,
    score_field = "score",
    exclude_items=[{"column": "genres", "values": ["Comedy|Romance", "Drama|Romance"]}],
    num_items=10,
)
recommendations_excluded_df = pd.DataFrame(recommendations_excluded).merge(movies, on="movie_id")
recommendations_excluded_df

### Item restriction

You can also restrict the list of recommendations to certain items using the keyword `restrict_items`. 

The input is a list of dictionaries where the format of each dictionary is as follows: {"column": "col0", "values": ["value0", "value1", "value3", ...]}

The command below is returning only comedies and dramas in the user recommendation list.

In [ ]:
recommendations_restricted = ApiClient().get_recommendations(
    deployment_token=deployment_token, 
    deployment_id=recommendations_deployment.deployment_id, 
    query_data=my_query_data,
    score_field = "score",
    restrict_items=[{"column": "genres", "values": ["Comedy", "Drama"]}],
    num_items=10,
)
recommendations_restricted_df = pd.DataFrame(recommendations_restricted).merge(movies, on="movie_id")
recommendations_restricted_df

### Compare the different user recommendations

We can compare the recommendation results with pandas commands.

In [ ]:
dataframe = pd.concat(
    [
        recommendations_base_df["movie"],
        recommendations_scaling_df["movie"],
        recommendations_excluded_df["movie"],
        recommendations_restricted_df["movie"],
    ],
    axis=1,
    ignore_index=True
)
dataframe.columns = ["Without Filter", "With Scaling Factor", "With Exclusion List", "With Restricted List"]
dataframe